In [37]:
import pandas as pd
import numpy as np
import random
import copy

In [38]:
df = pd.read_csv('./data/Football teams.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   team          98 non-null     object 
 1   tournament    98 non-null     object 
 2   goals         98 non-null     int64  
 3   shots_pg      98 non-null     float64
 4   yellow_cards  98 non-null     int64  
 5   red_cards     98 non-null     int64  
 6   possession%   98 non-null     float64
 7   pass%         98 non-null     float64
 8   aerialswon    98 non-null     float64
 9   rating        98 non-null     float64
dtypes: float64(5), int64(3), object(2)
memory usage: 7.8+ KB
None


,team,tournament,goals,shots_pg,yellow_cards,red_cards,possession%,pass%,aerialswon,rating
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01
1,Bayern Munich,Bundesliga,99,17.1,44,3,58.1,85.5,12.9,6.95
2,Paris Saint-Germain,Ligue 1,86,15.0,73,7,60.1,89.5,9.5,6.88
3,Barcelona,LaLiga,85,15.3,68,2,62.4,89.7,10.6,6.87
4,Real Madrid,LaLiga,67,14.4,57,2,57.7,87.7,11.8,6.86


In [39]:
df.tournament.unique()

array(['Premier League', 'Bundesliga', 'Ligue 1', 'LaLiga', 'Serie A'],
      dtype=object)

In [40]:
df.tournament.value_counts()

Premier League    20
Ligue 1           20
LaLiga            20
Serie A           20
Bundesliga        18
Name: tournament, dtype: int64

In [41]:
df[['team', 'tournament']].value_counts()

team                     tournament    
AC Milan                 Serie A           1
Rennes                   Ligue 1           1
Real Valladolid          LaLiga            1
Real Sociedad            LaLiga            1
Real Madrid              LaLiga            1
                                          ..
Elche                    LaLiga            1
Eintracht Frankfurt      Bundesliga        1
Eibar                    LaLiga            1
Dijon                    Ligue 1           1
Wolverhampton Wanderers  Premier League    1
Length: 98, dtype: int64

In [42]:
leagues_dict = {'Premier League': 'England', 'Bundesliga': 'Germany', 'Ligue 1': 'France', 'Serie A': 'Italy', 
                'LaLiga': 'Spain'}
df['country'] = df['tournament'].map(leagues_dict)
df.head()

,team,tournament,goals,shots_pg,yellow_cards,red_cards,possession%,pass%,aerialswon,rating,country
0,Manchester City,Premier League,83,15.8,46,2,60.8,89.4,12.8,7.01,England
1,Bayern Munich,Bundesliga,99,17.1,44,3,58.1,85.5,12.9,6.95,Germany
2,Paris Saint-Germain,Ligue 1,86,15.0,73,7,60.1,89.5,9.5,6.88,France
3,Barcelona,LaLiga,85,15.3,68,2,62.4,89.7,10.6,6.87,Spain
4,Real Madrid,LaLiga,67,14.4,57,2,57.7,87.7,11.8,6.86,Spain


In [43]:
df = df[['team', 'tournament', 'country']]
df.head()

,team,tournament,country
0,Manchester City,Premier League,England
1,Bayern Munich,Bundesliga,Germany
2,Paris Saint-Germain,Ligue 1,France
3,Barcelona,LaLiga,Spain
4,Real Madrid,LaLiga,Spain


In [44]:
df_cross = df.merge(df, on='country')
df_cross = df_cross.loc[df_cross.team_x!=df_cross.team_y, :]
df_cross.drop('tournament_y', inplace=True, axis=1)
df_cross.rename(columns={'team_x': 'home_team', 'team_y': 'away_team', 'tournament_x': 'tournament'}, inplace=True)
df_cross = df_cross[['home_team', 'away_team', 'tournament', 'country']]
df_cross

,home_team,away_team,tournament,country
1,Manchester City,Manchester United,Premier League,England
2,Manchester City,Aston Villa,Premier League,England
3,Manchester City,Chelsea,Premier League,England
4,Manchester City,Liverpool,Premier League,England
5,Manchester City,Tottenham,Premier League,England
...,...,...,...,...
1918,Benevento,Genoa,Serie A,Italy
1919,Benevento,Fiorentina,Serie A,Italy
1920,Benevento,Spezia,Serie A,Italy
1921,Benevento,Parma Calcio 1913,Serie A,Italy


In [45]:
df_cross = df_cross.sample(frac=1).sort_values(by='tournament').reset_index(drop=True)
df_cross

,home_team,away_team,tournament,country
0,Arminia Bielefeld,Bayern Munich,Bundesliga,Germany
1,Mainz 05,Hertha Berlin,Bundesliga,Germany
2,Bayern Munich,Freiburg,Bundesliga,Germany
3,Borussia M.Gladbach,Mainz 05,Bundesliga,Germany
4,VfB Stuttgart,Borussia Dortmund,Bundesliga,Germany
...,...,...,...,...
1821,Torino,Udinese,Serie A,Italy
1822,Cagliari,Bologna,Serie A,Italy
1823,Udinese,Napoli,Serie A,Italy
1824,Fiorentina,Napoli,Serie A,Italy


In [46]:
df_cross['tournament'].value_counts()

LaLiga            380
Ligue 1           380
Premier League    380
Serie A           380
Bundesliga        306
Name: tournament, dtype: int64

In [47]:
minutes_df = pd.DataFrame(np.arange(0, 91, 1), columns=['minute'])
minutes_df.head()

,minute
0,0
1,1
2,2
3,3
4,4


In [48]:
df_cross.head()

,home_team,away_team,tournament,country
0,Arminia Bielefeld,Bayern Munich,Bundesliga,Germany
1,Mainz 05,Hertha Berlin,Bundesliga,Germany
2,Bayern Munich,Freiburg,Bundesliga,Germany
3,Borussia M.Gladbach,Mainz 05,Bundesliga,Germany
4,VfB Stuttgart,Borussia Dortmund,Bundesliga,Germany


In [49]:
month = pd.DataFrame(np.arange(1, 13, 1), columns=['month'])
day = pd.DataFrame(np.arange(1, 32, 1), columns=['day'])
date_df = month.merge(day, how='cross')
date_df = date_df.loc[~(((date_df.month==2) & (date_df.day>28)) | 
                        ((date_df.month.isin([4, 6, 9, 11])) & (date_df.day>30))), :].reset_index()
date_df

,index,month,day
0,0,1,1
1,1,1,2
2,2,1,3
3,3,1,4
4,4,1,5
...,...,...,...
360,367,12,27
361,368,12,28
362,369,12,29
363,370,12,30


In [50]:
taken_dict = {}
df_list = []
for year in range(2000, 2024, 1):
    df_out = copy.deepcopy(df_cross)
    df_out['year'] = year
    taken_dict = {}
    for idx, row in df_cross.iterrows():
        if row['home_team'] not in taken_dict:
            taken_dict[row['home_team']] = set()
        if row['away_team'] not in taken_dict:
            taken_dict[row['away_team']] = set()
        
        date = date_df.sample(1).copy()
        
        while (int(date['month']), int(date['day'])) in taken_dict[row['home_team']] or (int(date['month']), int(date['day'])) in taken_dict[row['away_team']]:
            date = date_df.sample(1).copy()
        df_out.loc[idx, 'month'] = int(date['month'])
        df_out.loc[idx, 'day'] = int(date['day'])

        
        taken_dict[row['home_team']].add((int(date['month']), int(date['day'])))
        taken_dict[row['away_team']].add((int(date['month']), int(date['day'])))
    df_list.append(df_out.merge(minutes_df, how='cross'))

df_out = pd.concat(df_list)
print('done')

done


In [51]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3987984 entries, 0 to 166165
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   home_team   object 
 1   away_team   object 
 2   tournament  object 
 3   country     object 
 4   year        int64  
 5   month       float64
 6   day         float64
 7   minute      int32  
dtypes: float64(2), int32(1), int64(1), object(4)
memory usage: 258.6+ MB


In [52]:
df_out['home_score'] = np.random.choice([0, 1], size=len(df_out), p=[0.99, 0.01])
df_out['away_score'] = np.random.choice([0, 1], size=len(df_out), p=[0.99, 0.01])
df_out.loc[df_out.minute==0, 'home_score'] = 0
df_out.loc[df_out.minute==0, 'away_score'] = 0

In [53]:
df_out['home_score'] = df_out.groupby(list(df_out.columns.drop(['minute', 'home_score', 
                                                                 'away_score'])))['home_score'].cumsum()
df_out['away_score'] = df_out.groupby(list(df_out.columns.drop(['minute', 'home_score', 
                                                                 'away_score'])))['away_score'].cumsum()

In [54]:
df_out.tail(90)

,home_team,away_team,tournament,country,year,month,day,minute,home_score,away_score
166076,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,1,0,0
166077,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,2,0,0
166078,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,3,0,0
166079,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,4,0,0
166080,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,5,0,0
...,...,...,...,...,...,...,...,...,...,...
166161,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,86,0,0
166162,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,87,0,0
166163,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,88,0,0
166164,Crotone,Fiorentina,Serie A,Italy,2023,1.0,14.0,89,0,0


In [55]:
df_out.to_csv('./data/scores_output.csv', index=False)

In [56]:
df = pd.read_csv('./data/scores_output.csv')
df.head()

,home_team,away_team,tournament,country,year,month,day,minute,home_score,away_score
0,Arminia Bielefeld,Bayern Munich,Bundesliga,Germany,2000,4.0,22.0,0,0,0
1,Arminia Bielefeld,Bayern Munich,Bundesliga,Germany,2000,4.0,22.0,1,0,0
2,Arminia Bielefeld,Bayern Munich,Bundesliga,Germany,2000,4.0,22.0,2,0,0
3,Arminia Bielefeld,Bayern Munich,Bundesliga,Germany,2000,4.0,22.0,3,0,0
4,Arminia Bielefeld,Bayern Munich,Bundesliga,Germany,2000,4.0,22.0,4,0,0


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3987984 entries, 0 to 3987983
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   home_team   object 
 1   away_team   object 
 2   tournament  object 
 3   country     object 
 4   year        int64  
 5   month       float64
 6   day         float64
 7   minute      int64  
 8   home_score  int64  
 9   away_score  int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 304.3+ MB
